In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

# Flight Dynamics
## Turning
### Bank angle and structural loading (g-loading $n$)
in a coordinated turn

$$R = \frac{v^2}{g \dot \tan \theta} = \frac{v^2}{g \sqrt{n^2 - 1}}$$
$$t = \frac{2 \pi v}{g \sqrt{n^2 - 1}}$$

In [ ]:
inch_to_m = 0.0254  # [=] m/inch
feet_to_m = 0.3048  # [=] m/ft
lb_to_kg = 0.453592  # [=] kg/lb
g = 9.81  # [=] m/s^2

bank_angles = np.linspace(1, 80, 80)
g_loadings = 1 / np.cos(np.deg2rad(bank_angles))

v = 100 * feet_to_m
Rs = v**2 / g / np.tan(np.deg2rad(bank_angles))
turning_time = 2 * np.pi * Rs / v

# Make two y axes
fig, g_load_axis = plt.subplots()
g_load_axis.plot(bank_angles, g_loadings, label="G loading", color="tab:orange")
g_load_axis.set_xlabel("Bank angle (°)")
g_load_axis.set_ylabel("G Loading")

turn_radius_axis = g_load_axis.twinx()
turn_radius_axis.plot(bank_angles, Rs, label="Turn Radius")
turn_radius_axis.set_ylabel("Turn radius")
turn_radius_axis.set_ylim([0, 500])

# plt.plot(bank_angles, g_loadings, label= "G Loading")
# plt.plot(bank_angles, Rs)
# plt.plot(bank_angles, Rs)

# plt.xlabel('Bank angle [deg]')
# plt.ylabel('g loading')
fig.legend()
plt.show()

In [ ]:
g_loading_max = 4
speeds = np.linspace(10, 500, 100) * feet_to_m
turning_times = 2 * np.pi * speeds / g / np.sqrt(g_loading_max**2 - 1)
lap_times = 1000 * 2 * feet_to_m / speeds + turning_times

# make two y axis
fig, ax1 = plt.subplots()
ax1.plot(speeds / feet_to_m, lap_times)
ax1.set_xlabel("Speed [ft/s]")
ax1.set_ylabel("Lap time [s]")


ax2 = ax1.twinx()
ax2.plot(speeds / feet_to_m, 300 / lap_times, color="tab:orange")
ax2.set_ylabel("Laps in 5 minutes")
plt.show()


# DBF Missions
Refer to the AIAA DBF page for a comprehensive set of rules:\
[AIAA DBF](https://aiaa.org/dbf/)

## Mission 2 Rules
#### Payload Rules
- **Passengers**: 2.3-inch rubber ducks (~0.7 oz each).
- **Cargo**: Standard hockey pucks (6 oz each).
- **Payload Ratio**: The airplane must be capable of carrying at least **3 passengers** for each **1 piece of cargo**.
$$N_{\text{cargo}}=\left\lfloor\frac{N_\text{passengers}}3\right\rfloor$$

#### Mission 2 Scoring: Net Income
The score for Mission 2 is the "Net Income" generated from a charter flight within a 5-minute window.

**Net Income Equation:**
$$\text{Net Income}=\text{Income}-\text{Cost}$$

**1. Income Calculation:**
Income is based on the number of passengers/cargo and the number of laps flown.

$$\text{Income}=N_\text{passengers}\times(I_{\text{passenger,fixed}} + (I_{\text{passenger,lap}} \times N_\text{laps})) + N_\text{cargo}\times(I_\text{cargo,fixed} + (I_\text{cargo,lap} \times N_\text{laps}))$$
| Variable | Significance | Value |
| -- | -- | -- |
|$I_\text{passenger,fixed}$|Fixed income per passenger|6|
|$I_\text{passenger,lap}$|Income per passenger per lap|2|
|$I_\text{cargo,fixed}$|Fixed income per cargo|10|
|$I_\text{cargo,lap}$|Income per cargo per lap|8|

**2. Cost Calculation:**
Cost is based on a base operating cost plus costs for passengers/cargo, adjusted by an Efficiency Factor (EF).

$$\text{Cost}=N_\text{laps}\times(C_\text{base,lap} + N_\text{passengers}\times C_\text{passenger,lap} + N_\text{cargo}\times C_\text{cargo,lap})\times e$$
| Variable | Significance | Value |
| -- | -- | -- |
|$C_\text{base,lap}$|Base operating cost per lap|10|
|$C_\text{passenger,lap}$|Passenger operating cost per lap|0.5|
|$C_\text{cargo,lap}$|Cargo operating cost per lap|2|
|$e$|Efficiency Factor This penalizes higher-capacity batteries.|$e=\frac{\text{Total battery capacity}}{100\mathrm{Wh}}$|


In [ ]:
# --- Constants based on the DBF Rules---

# Passenger (Duck) Properties
DUCK_WEIGHT_OZ = 0.7  # Using max weight for a conservative estimate (0.6-0.7 oz)
DUCK_WIDTH_IN = 2.3
DUCK_LENGTH_IN = 2.5
DUCK_HEIGHT_IN = 2.5
DUCK_VOLUME_IN3 = DUCK_WIDTH_IN * DUCK_LENGTH_IN * DUCK_HEIGHT_IN

# Cargo (Hockey Puck) Properties
PUCK_WEIGHT_OZ = 6.0
PUCK_DIAMETER_IN = 3.0
PUCK_THICKNESS_IN = 1.0
PUCK_VOLUME_IN3 = math.pi * ((PUCK_DIAMETER_IN / 2) ** 2) * PUCK_THICKNESS_IN

# Mission 2 Scoring Parameters
INCOME_FIXED_PER_PASSENGER = 6
INCOME_PER_PASSENGER_PER_LAP = 2
INCOME_FIXED_PER_CARGO = 10
INCOME_PER_CARGO_PER_LAP = 8

COST_BASE_PER_LAP = 10
COST_PER_PASSENGER_PER_LAP = 0.5
COST_PER_CARGO_PER_LAP = 2

In [ ]:
def payload_properties(num_passengers, num_cargo=None):
    """
    Calculates the total weight and estimated volume for passengers and associated cargo.

    Args:
        num_passengers (int): The number of passengers (ducks).

    Returns:
        dict: A dictionary containing the number of cargo pieces, total weight in ounces,
              and estimated total volume in cubic inches. Returns None if num_passengers < 3.
    """
    # Although the plane cargo capacity must be set to the following
    # constraints, Mission 2 loading does not have any constraints

    if num_passengers < 3:
        # Although the plane must be *capable* of carrying 3, a flight may have 0.
        '''
        raise Warning(
            "DBF RULE VIOLATION: MINIMUM PASSENGER COUNT\nThere must be a minimum of 3 passengers. {num_passengers} passengers requested"
        )
        '''
    
    if num_cargo is None:
        # If not cargo is specified assume max possible cargo loading
        num_cargo = np.floor(num_passengers / 3)
    elif num_cargo * 3 > num_passengers:
        '''
        raise Warning(
            "DBF RULE VIOLATION: CARGO-PASSENGER RATIO\nThere must be a minimum of {} passengers for {} cargo. {} cargo and {} passengers requested".format(
                num_cargo * 3,
                num_cargo,
                num_cargo,
                num_passengers,
            )
        )
        '''

    # Minimum design constraints by volume
    min_passenger_volume = np.max([
        3 * DUCK_VOLUME_IN3, # Minimum 3 passengers
        num_cargo * 3 * DUCK_VOLUME_IN3, # Minimum 3 passengers per cargo
    ])

    total_weight_oz = (
        (num_passengers * DUCK_WEIGHT_OZ) + (num_cargo * PUCK_WEIGHT_OZ)
    )

    total_volume_in3 = (
        max((num_passengers * DUCK_VOLUME_IN3), min_passenger_volume) +
        (num_cargo * PUCK_VOLUME_IN3)
    )

    return {
        "num_pax": num_passengers,
        "num_cargo": num_cargo,
        "weight_oz": total_weight_oz,
        "estimated_volume_in3": total_volume_in3,
    }

In [ ]:
def mission2_score(
    num_passengers,
    num_cargo,
    num_laps,
    battery_capacity_wh=100.0,
):
    """
    Calculates the Net Income for Mission 2 based on the rules.

    Args:
        num_passengers (int): The number of passengers (ducks) flown.
        num_laps (int): The number of laps completed.
        battery_capacity_wh (float, optional): The total propulsion battery capacity in Watt-hours. Defaults to 100.0.

    Returns:
        float: The calculated Net Income for the mission.
    """
    # The minimum capacity of the airplane is 3 passengers and 1 cargo.
    # However, a team can choose to fly with fewer or no payload.
    if num_passengers > 0:
        payload_props = payload_properties(num_passengers, num_cargo)
        num_cargo = payload_props["num_cargo"]
    else:  # No passengers means no cargo either for scoring flight
        num_cargo = 0

    # Calculate total income
    income = (num_passengers * (INCOME_FIXED_PER_PASSENGER + (INCOME_PER_PASSENGER_PER_LAP * num_laps))) + (num_cargo * (INCOME_FIXED_PER_CARGO + (INCOME_PER_CARGO_PER_LAP * num_laps)))

    # Calculate total cost
    efficiency_factor = battery_capacity_wh / 100.0
    cost_per_lap = COST_BASE_PER_LAP + (num_passengers * COST_PER_PASSENGER_PER_LAP) + (num_cargo * COST_PER_CARGO_PER_LAP)
    cost = num_laps * cost_per_lap * efficiency_factor

    net_income = income - cost
    return net_income


In [ ]:
example_passengers = 50 
example_cargo = None
example_laps = 5

# Calculate payload weight and volume
payload = payload_properties(example_passengers, example_cargo)
print(f"--- Payload Properties for {example_passengers} Passengers ---")
print(f"Number of Cargo Pieces: {payload['num_cargo']}")
print(f"Total Weight: {payload['weight_oz']:.2f} oz")
print(f"Estimated Volume: {payload['estimated_volume_in3']:.2f} in³\n")

score = mission2_score(example_passengers, example_cargo, example_laps)
print("--- Score Calculation Example ---")
print(f"Net Income for {example_passengers} passengers and {example_laps} laps: {score:.2f}\n")

In [ ]:
# Calculate properties for a range of passengers and cargo

max_pax = 50
max_cargo = 20

pax_range = np.arange(0, max_pax + 1)
cargo_range = np.arange(0, max_cargo + 1)

# print(len(pax_range))
# print(len(cargo_range))

passengers, cargos = np.meshgrid(pax_range, cargo_range)
properties = np.zeros_like(passengers, dtype=dict) # Create an "empty" matrix

for x, num_passengers in enumerate(pax_range):
    for y, num_cargos in enumerate(cargo_range):
        properties[y, x] = payload_properties(
            num_passengers,
            num_cargos,
        )
        # print(x, y)
        # print(properties[x, y])
        
# print(len(properties))
# print(len(properties[0]))
# print(properties)

In [ ]:
def extract_properties(properties, property_key):
    """
    Gets a 2d array of properties and returns a 2d array of values of a property
    
    Args:
        properties (dict[][]): A 2d array of values returned from the function payload_properties
        property_key (str): A string that represents the property to be retrieved
        
    Returns:
        Some[][]: A 2d array of values of properties
    """
    
    values = np.zeros_like(properties)

    for i, property_array in enumerate(properties):
        for j, property in enumerate(property_array):
            # print(property)
            # print(property[property_key])
            values[i, j] = property[property_key]
            
    return values

In [ ]:
# Extract weights
weights = extract_properties(properties, 'weight_oz').astype(float)

# Plot weight contour
plt.figure(figsize=(12, 8))
# Filled contour plot
contour = plt.contourf(passengers, cargos, weights, levels=20, cmap="viridis")

# Add contour lines with labels
line_contour = plt.contour(
    passengers, cargos, weights,
    levels=contour.levels,
    colors='white',
    linewidths=0.5,
)
plt.clabel(line_contour, inline=True, fontsize=8, fmt='%1.0f')

plt.colorbar(contour, label='Payload weight [oz]')
plt.title('Payload weight by number of passengers/cargo')
plt.locator_params(axis="both", integer=True)
plt.plot(pax_range, pax_range/3, color='red', label='Max design cargo')
plt.legend()
plt.xlabel('Passengers')
plt.ylabel('Cargo')
plt.grid(True, ls='--', alpha=0.6)
plt.show()

In [ ]:
# Extract volumes
volumes = extract_properties(properties, 'estimated_volume_in3').astype(float)

# Plot volume contour
plt.figure(figsize=(12, 8))
# Filled contour plot
contour = plt.contourf(passengers, cargos, volumes, levels=20, cmap="viridis")

# Add contour lines with labels
line_contour = plt.contour(
    passengers, cargos, volumes,
    levels=contour.levels,
    colors='white',
    linewidths=0.5,
)
plt.clabel(line_contour, inline=True, fontsize=8, fmt='%1.0f')

plt.colorbar(contour, label='Required cargo space [in³]')
plt.title('Required cargo space by number of passengers/cargo')
plt.locator_params(axis="both", integer=True)
plt.plot(pax_range, pax_range/3, color='red', label='Max design cargo')
plt.legend()
plt.xlabel('Passengers')
plt.ylabel('Cargo')
plt.grid(True, ls='--', alpha=0.6)
plt.show()

In [ ]:
# Plot score contour
plt.figure(figsize=(12, 8))
# Filled contour plot
contour = plt.contourf(passengers, cargos, scores,
                       levels=20, cmap="viridis")

# Add contour lines with labels
line_contour = plt.contour(
    passengers, cargos, scores,
    levels=contour.levels,
    colors='white',
    linewidths=0.5,
)
plt.clabel(line_contour, inline=True, fontsize=8, fmt='%1.0f')

plt.colorbar(contour, label='Net income')
plt.title('Net income by passengers/cargo with 6 laps and a 100Wh battery')
plt.locator_params(axis="both", integer=True)
plt.plot(pax_range, pax_range/3, color='red', label='Max design cargo')
plt.legend()
plt.xlabel('Passengers')
plt.ylabel('Cargo')
plt.grid(True, ls='--', alpha=0.6)
plt.show()

In [ ]:
# Plot score/weight contour
plt.figure(figsize=(12, 8))
# Filled contour plot
score_weight_efficiency = scores/weights
contour = plt.contourf(passengers, cargos, score_weight_efficiency,
                       levels=20, cmap="viridis")

# Add contour lines with labels
line_contour = plt.contour(
    passengers, cargos, score_weight_efficiency,
    levels=contour.levels,
    colors='white',
    linewidths=0.5,
)
plt.clabel(line_contour, inline=True, fontsize=8, fmt='%1.2f')

plt.colorbar(contour, label='Income-weight efficiency [oz^(-1)]')
plt.title('Net income-weight efficiency by passengers/cargo with 6 laps and a 100Wh battery')
plt.locator_params(axis="both", integer=True)
plt.plot(pax_range, pax_range/3, color='red', label='Max design cargo')
plt.legend()
plt.xlabel('Passengers')
plt.ylabel('Cargo')
plt.grid(True, ls='--', alpha=0.6)
plt.show()

In [ ]:
# Plot score/volume contour
plt.figure(figsize=(12, 8))
# Filled contour plot
score_volume_efficiency = scores/volumes
contour = plt.contourf(passengers, cargos, score_volume_efficiency,
                       levels=20, cmap="viridis")

# Add contour lines with labels
line_contour = plt.contour(
    passengers, cargos, score_volume_efficiency,
    levels=contour.levels,
    colors='white',
    linewidths=0.5,
)
plt.clabel(line_contour, inline=True, fontsize=8, fmt='%1.2f')

plt.colorbar(contour, label='Income-volume efficiency [in^(-3)]')
plt.title('Net income-volume efficiency by passengers/cargo with 6 laps and a 100Wh battery')
plt.locator_params(axis="both", integer=True)
plt.plot(pax_range, pax_range/3, color='red', label='Max design cargo')
plt.legend()
plt.xlabel('Passengers')
plt.ylabel('Cargo')
plt.grid(True, ls='--', alpha=0.6)
plt.show()